### Connect Database

In [ ]:
import pyodbc
server = 'group15-server.database.windows.net'  
database = 'group15_Yelp_Database'  
username = 'group15'  
password = 'Badam15123'  
driver = '{ODBC Driver 18 for SQL Server}' 
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};'
    'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=60;'
)
cursor = conn.cursor()
print("Connected.")

Connected.


In [4]:
# Run Query Function
import pandas as pd
def run_query(conn, sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(rows, columns=columns)

# Usage:
#df = run_query(conn, query)

## Milestone 3

1. Target City: Philadelphia

Philadelphia is a major East Coast hub with strong commuter and visitor flows via I‑95, I‑76, Amtrak, and regional transit, creating consistent foot traffic and discovery opportunities.

The city also has one of the largest Polish communities in the United States—and the largest in Pennsylvania—providing a built-in customer base that values authentic cuisine and cultural connection.

Philadelphia’s culinary scene is vibrant yet leaves room for niche ethnic concepts, presenting an opportunity to differentiate by serving underserved tastes and community representation.

2. Restaurant Concept

- Cuisine: Polish—offering authentic dishes such as pierogi, kielbasa, bigos, gołąbki, żurek, and pączki.
- Price Point: Individual meals under $20 to stay accessible for students, families, and commuters.
- Style: Fast-casual hybrid—optimize for speed and value for on-the-go diners, with a cozy area for locals to relax and connect.

3. Target Customers

Focus on Philadelphia locals—especially the Polish community—to build steady, repeat business and cultural authenticity.

Tourists and commuters moving through Philadelphia’s transit corridors represent transient demand, adding diversity and new-customer exposure. Proximity to major highways and transit hubs increases visibility and convenience.

Both groups value accessible pricing, authentic food, and efficient service, turning a quick meal into a memorable cultural experience.

In [ ]:
# 1. Count of Polish Restaurants in Philadelphia
polish_restaurant_count_query = """
SELECT COUNT(*) AS polish_restaurant_count
FROM business
WHERE city = 'Philadelphia' AND state = 'PA' AND categories LIKE '%Polish%';
"""
polish_restaurant_count_df = run_query(conn, polish_restaurant_count_query)
polish_restaurant_count_df.head()

Context: Reveals saturation. If the count is low, there’s a strong niche opportunity.

In [ ]:
# 2. Average Rating and Reviews for Polish Restaurants in Philadelphia
avg_rating_reviews_query = """
SELECT AVG(stars) AS avgrating, AVG(review_count) AS avgreviews
FROM business
WHERE city = 'Philadelphia' AND state = 'PA' AND categories LIKE '%Polish%';
"""
avg_rating_reviews_df = run_query(conn, avg_rating_reviews_query)
avg_rating_reviews_df.head()


Context: Benchmarks customer satisfaction and engagement, helping set goals for quality and visibility.

In [ ]:
# 3. Distribution of Polish Restaurant Ratings in Philadelphia
rating_distribution_query = """
SELECT stars, COUNT(*) AS restaurantcount
FROM business
WHERE city = 'Philadelphia' AND state = 'PA' AND categories LIKE '%Polish%'
GROUP BY stars
ORDER BY stars DESC;
"""
rating_distribution_df = run_query(conn, rating_distribution_query)
rating_distribution_df.head()

Context: Reveals trends in quality—are most Polish restaurants highly rated or do they struggle?

In [ ]:
# 4. Neighborhoods with No Polish Restaurants in Philadelphia
no_polish_restaurants_query = """
SELECT DISTINCT b.address
FROM business b
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Restaurant%'
AND b.address NOT IN (
  SELECT address
  FROM business
  WHERE city = 'Philadelphia' AND state = 'PA' AND categories LIKE '%Polish%'
);
"""
no_polish_restaurants_df = run_query(conn, no_polish_restaurants_query)
no_polish_restaurants_df.head()


Context: Identifies “restaurant deserts” suitable for a new Polish restaurant.

In [ ]:
# 5. Most Valued Attributes Among Top-Rated Polish Restaurants in Philadelphia
top_attributes_query = """
SELECT a.attribute_key, COUNT(*) AS freq
FROM business b
JOIN attributes a ON b.business_id = a.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.stars >= 4 AND b.categories LIKE '%Polish%'
GROUP BY a.attribute_key
ORDER BY freq DESC;
"""
top_attributes_df = run_query(conn, top_attributes_query)
top_attributes_df.head()

Context: Prioritizes operational features (like “Takes Reservations”) based on what works.



In [ ]:
# 6. Common Complaints in 1-2 Star Reviews of Polish Restaurants in Philadelphia
common_complaints_query = """
SELECT COUNT(*) AS complaintcount
FROM review r
JOIN business b ON r.business_id = b.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%'
AND r.stars <= 2
AND (r.text LIKE '%wait%' OR r.text LIKE '%price%' OR r.text LIKE '%service%' OR r.text LIKE '%portion%');
"""
common_complaints_df = run_query(conn, common_complaints_query)
common_complaints_df.head()

In [ ]:
Context: Reveals recurring service or food issues to avoid.

In [ ]:
# 7. Top Keywords in Positive 4 Star Reviews for Polish Restaurants in Philadelphia
top_keywords_query = """
SELECT TOP 10 r.text, COUNT(*) AS freq
FROM review r
JOIN business b ON r.business_id = b.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%' AND r.stars = 4
GROUP BY r.text
ORDER BY freq DESC;
"""
top_keywords_df = run_query(conn, top_keywords_query)
top_keywords_df.head()

Context: Determines which aspects (food, staff, atmosphere) customers appreciate most.



In [ ]:
# 8. Polish Restaurant Sub-Categories with Few Competitors, High Ratings in Philadelphia
sub_categories_query = """
SELECT categories, COUNT(*) AS restaurantcount, AVG(stars) AS avgrating
FROM business
WHERE city = 'Philadelphia' AND state = 'PA' AND categories LIKE '%Polish%'
GROUP BY categories
HAVING COUNT(*) <= 3 AND AVG(stars) >= 4
ORDER BY avgrating DESC;
"""
sub_categories_df = run_query(conn, sub_categories_query)
sub_categories_df.head()

Context: Finds possible segment “gaps” for specialization.

In [ ]:

# 9. Peak Check-in Times for Polish Restaurants in Philadelphia
peak_checkin_query = """
SELECT h.day, h.open_time, h.close_time, COUNT(c.business_id) AS checkincount
FROM business b
JOIN hours h ON b.business_id = h.business_id
JOIN checkin c ON b.business_id = c.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%'
GROUP BY h.day, h.open_time, h.close_time
ORDER BY checkincount DESC;
"""
peak_checkin_df = run_query(conn, peak_checkin_query)
peak_checkin_df.head()

Context: Guides staffing, service style, and promotional timing.

In [ ]:

# 10. Longevity and Rating of Polish Restaurants in Philadelphia
longevity_rating_query = """
SELECT b.name, MIN(r.date) AS opensince, AVG(r.stars) AS avgrating
FROM business b
JOIN review r ON b.business_id = r.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%'
GROUP BY b.name
ORDER BY opensince ASC;
"""
longevity_rating_df = run_query(conn, longevity_rating_query)
longevity_rating_df.head()

Context: Understands longevity and sets expectations for performance as the business matures.

In [ ]:
# 11. Busiest Days for Polish Restaurants by Check-ins in Philadelphia
busiest_days_query = """
SELECT h.day, COUNT(c.business_id) AS totalcheckins
FROM business b
JOIN hours h ON b.business_id = h.business_id
JOIN checkin c ON b.business_id = c.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%'
GROUP BY h.day
ORDER BY totalcheckins DESC;
"""
busiest_days_df = run_query(conn, busiest_days_query)
busiest_days_df.head()

Context: Pinpoints which days see the highest customer volume, guiding weekly specials or staffing plans.

In [ ]:

# 12. Average Review Length and Average Rating for Polish Restaurants in Philadelphia
avg_review_length_query = """
SELECT AVG(LEN(r.text)) AS avgreviewlength, AVG(r.stars) AS avgrating
FROM review r
JOIN business b ON r.business_id = b.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%';
"""
avg_review_length_df = run_query(conn, avg_review_length_query)
avg_review_length_df.head()


Context: Longer reviews may indicate deeper engagement (positive or negative); helps interpret feedback quality.



In [ ]:
# 13. Currently Open vs Closed Polish Restaurants in Philadelphia
open_vs_closed_query = """
SELECT is_open, COUNT(*) AS restaurantcount
FROM business
WHERE city = 'Philadelphia' AND state = 'PA' AND categories LIKE '%Polish%'
GROUP BY is_open;
"""
open_vs_closed_df = run_query(conn, open_vs_closed_query)
open_vs_closed_df.head()


Context: Shows current market activity and potential risk (e.g., closures may indicate obstacles in the niche).

In [ ]:
# 14. Attributes Linked to Closed Polish Restaurants in Philadelphia
closed_attributes_query = """
SELECT a.attribute_key, COUNT(*) AS freq
FROM business b
JOIN attributes a ON b.business_id = a.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%' AND b.is_open = 0
GROUP BY a.attribute_key
ORDER BY freq DESC;
"""
closed_attributes_df = run_query(conn, closed_attributes_query)
closed_attributes_df.head()


Context: Reveals whether specific features (e.g., lack of kid-friendliness, poor accessibility) are linked to unsuccessful restaurants.

In [ ]:
# 15. Average Days Since Last Check-in for Polish Restaurants in Philadelphia
avg_days_since_checkin_query = """
SELECT AVG(DATEDIFF(day, CAST(c.date AS DATETIME), GETDATE())) AS avgdayssincelastcheckin
FROM business b
JOIN checkin c ON b.business_id = c.business_id
WHERE b.city = 'Philadelphia' AND b.state = 'PA' AND b.categories LIKE '%Polish%';
"""
avg_days_since_checkin_df = run_query(conn, avg_days_since_checkin_query)
avg_days_since_checkin_df.head()


Context: Measures current customer engagement—recent check-ins indicate active demand, while long intervals signal decline.